In [1]:
import numpy as np
import pandas as pd
from os import path
from ete3 import Tree, TreeStyle, NodeStyle
from PIL import Image

In [2]:
# Read trees
t0 = Tree("../data/gtt/gtt_cyanobacteria/gtt_cyanobacteria.tre")
t1 = Tree("../data/gtt/gtt_bacteria/gtt_bacteria.tre")

t0_leaves = 0
t1_leaves = 0

for node in t0.traverse():
    if node.is_leaf():
        t0_leaves += 1

for node in t1.traverse():
    if node.is_leaf():
        t1_leaves += 1
        
t0_leaves, t1_leaves

# I ended up not using the Universal marker set.
# t = Tree("../data/gtt/gtt_universal/gtt_universal.tre")

(162, 160)

In [3]:
# Set colors
branco = NodeStyle()
azul1, azul2 = NodeStyle(), NodeStyle()
azul1["bgcolor"] = "#D3DBFE"
azul2["bgcolor"] = "#C0CBFD"
cyan1, cyan2, cyan3 = NodeStyle(), NodeStyle(), NodeStyle()
cyan1["bgcolor"] = "#BFE0E9"
cyan2["bgcolor"] = "#D4E9EF"
green1 = NodeStyle()
green1["bgcolor"] = "#D0D9C9"
purple1, purple2 = NodeStyle(), NodeStyle()
purple1["bgcolor"] = "#DBC7E4"
purple2["bgcolor"] = "#E6D9EC"

# Function to enlarge branches
def make_branches_bigger(node, new_size):
    node.img_style["hz_line_width"] = new_size # Change the horizotal lines stroke size
    node.img_style["vt_line_width"] = new_size # Change the vertical lines stroke size
    for c in node.children:
        make_branches_bigger(c, new_size)

# Set our TreeStyle
c = TreeStyle()
c.mode = "r"
c.scale = 120
c.show_branch_length = True
c.show_branch_support = True

# Traverse each tree and apply Tree Styles, NodeStyle
for t in (t0, t1):
    make_branches_bigger(t, 5)
    for node in t.traverse():
        if node.is_leaf():
            node.name = node.name.replace("_", " ")
            if "nan" in node.name:
                print(node.name)
                node.delete()
            if "Pseudosynechococcus" in node.name:
                node.set_style(azul1)
            elif "Parasynechococcus" in node.name:
                node.set_style(azul2)
            elif "Magnicoccus" in node.name:
                node.set_style(cyan1)
            elif "Regnicoccus" in node.name:
                node.set_style(cyan2)
            elif "Lacustricoccus" in node.name:
                node.set_style(cyan2)
            elif "Inmanicoccus" in node.name:
                node.set_style(cyan1)
            elif "Synechospongium" in node.name:
                node.set_style(green1)   
            elif "Enugrolinea" in node.name:
                node.set_style(purple1)    
            elif "Synechococcus" in node.name:
                node.set_style(purple2)            
            elif "Leptococcus" in node.name:
                node.set_style(purple1)
            elif "Thermosynechococcus" in node.name:
                node.set_style(purple2)        
            elif "Prochlorococcus" in node.name:  # Our root
                t.set_outgroup(node)
                make_branches_bigger(node, 5)
                node.set_style(branco)
            elif any((s in node.name for s in ("Coccusdissimilis", "Leptovivax", "Stenotopis", "Brevicoccus", "Lividococcus"))):
                print(node.name)
                node.delete()
    t.ladderize()


# Render Trees
r = t0.render("../data/plots/Figure_3A_cyanobacterial.png", tree_style=c, h=2400, w=600, dpi=1000)
r = t1.render("../data/plots/Figure_3B_bacterial.png", tree_style=c, h=2400, w=600, dpi=1000)

In [4]:
# Combine trees with Pillow
imgs = ["../data/plots/Figure_3A_cyanobacterial.png",
          "../data/plots/Figure_3B_bacterial.png"]
imgs = [Image.open(i) for i in imgs]
imgs = np.hstack((np.asarray(i) for i in imgs))
imgs = Image.fromarray(imgs)
imgs.save("../data/plots/Figure_3_combined.png")

<ipython-input-4-0b98ffa84209>:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs = np.hstack((np.asarray(i) for i in imgs))
